In [2]:
import numpy as np
from scipy.io.wavfile import write

class SoundObj:
    def __init__(self, Fsample, duration):
        self.Fcs = {
            "C": 262.0,
            "D": 294.0,
            "E": 330.0,
            "F": 349.0,
            "G": 392.0,
            "A": 440.0,
            "B": 494.0,
        }

        self.Fs = Fsample
        delta = 1.0 / Fsample
        self.dur = duration
        self.N = int(duration * Fsample)  # 音の長さ
        self.t = np.arange(self.N) * delta  # 時間刻み
        self.x = np.zeros(self.N)  # 音信号

    def getSound(self):
        return self.x

    def getLen(self):
        return self.N

    def setTone(self, code="C", Amp=1.0):
        f = self.Fcs[code]
        self.x = Amp * np.sin(2.0 * np.pi * f * self.t)

    def normalize(self):

        xmax = np.abs(self.x).max()
        self.x /= xmax


    def __add__(self, other):
        if isinstance(other, self.__class__) and self.Fs == other.Fs:
            tmp = SoundObj(self.Fs, self.dur + other.dur)
            tmp.x = np.hstack((self.x, other.x))
            return tmp
        else: 
            raise NotImplementedError()

if __name__ == "__main__":
    Fs = 22100.0
    dur = 1.0
    fname = "OOPmksnd.wav"

    c = SoundObj(Fs, dur)
    c.setTone("C")

    e = SoundObj(Fs, dur)
    e.setTone("E")

    g = SoundObj(Fs, dur)
    g.setTone("G")

    s = c + e + g  # + は __add__ で実装

    # モノラルでファイル吐き出し
    # ここも float32 に変換して wav フォーマットに乗っけるようにしておく
    # 音の振幅が [-1, 1] になるように先に normalize() を呼んでおく
    s.normalize()
    write(fname, int(Fs), s.getSound().astype("float32"))